In [ ]:
from pathlib import Path
import pandas as pd
import shutil
import src.pic_preprocess as pp
import src.csv_analyser as analy

In [ ]:
input_file_names=["アノテーションDB_20250117-141645__ベアSi_0-正常_一覧",                  
"アノテーションDB_20250117-141645__ベアSi_1-汚れ_一覧",                 
"アノテーションDB_20250117-141645__ベアSi_2-擦り跡_一覧",                 
"アノテーションDB_20250117-141645__ベアSi_3-発塵痕_一覧",                 
"アノテーションDB_20250117-141645__ベアSi_4-打痕、傷_一覧",                 
"アノテーションDB_20250117-141645__ベアSi_5-異物_一覧",                 
"アノテーションDB_20250117-141645__ベアSi_6-ウォーターマーク_一覧",                 
"アノテーションDB_20250117-141645__ベアSi_7-残渣_一覧",                 
"アノテーションDB_20250117-141645__ベアSi_8-ヒゲ_一覧",                 
"アノテーションDB_20250117-141645__ベアSi_9-膜境界不良_一覧",                 
"アノテーションDB_20250117-141645__ベアSi_10-コメット_一覧",                 
"アノテーションDB_20250117-141645__ベアSi_11-タイガーストライプ_一覧",]

fieldnames = [
        "image_id", "original_id", "measurement_id", "foup_slot", "bevel_section",
        "flame_no", "split", "making_defect_type", "selection_no", "defect_class"
    ]
df_split_info = pd.DataFrame(columns=fieldnames)

for i in range(len(input_file_names)):
    input_file_name=input_file_names[i]
    excel_path = Path(r"C:\Users\huang\work\dataset\make_dataset\no_pattern") / f"{input_file_name}.csv"

    output_file_folder_1st=input_file_name.split("_")[3]
    output_file_folder_2nd=input_file_name.split("_")[4]
 
    src_dir = r"D:\H23057-J_とめ研社内共有\画像データ\オリジナル\20241004_画像データ\提出画像_20241003"
    dest_dir = Path(r"D:\H23057-J_とめ研社内共有\画像データ\アノテーション済\20250207_bareSi")
    dest_dir = dest_dir / output_file_folder_1st / output_file_folder_2nd

    category_names=["01_normal","02_blot","03_scratch","04_dust","05_dent","06_foreign_substance","07_watermark","08_residue","09_barb","10_defective_boundary","11_comet","12_tiger_stripe"]
    defect_class=category_names[i]

    df=pd.read_csv(excel_path,encoding='shift_jis', dtype=str,on_bad_lines='warn')   
    df['画像コード'] = df['画像コード'].str.replace('-', '_')
    
    if df.empty or df.shape[0] <= 1:
        print(f"Failed to load or no data except index row in {excel_path}")
        continue
       
    for image in df['画像コード']:
        is_vertical = "C" in image
        #image_id,original_id,measurement_id,bevel_section,flame_no,split=analy.process_image_name(image,is_vertical)  
        image_info = analy.process_image_name(image,is_vertical)           
        analy.find_and_copy_images(src_dir, dest_dir, image_info.original_id)  
              
    pp.split_images_in_folder(dest_dir, dest_dir)    
    for image in df['画像コード']: 
        tif_file=dest_dir/ "splitted"/ (image+ '.tif')
        if not (dest_dir.parent/"input_data").exists():
            (dest_dir.parent/"input_data") .mkdir(parents=True) 
        destination_file = dest_dir.parent/"input_data" / ( image+ '.tif')
        if (tif_file.exists()) and not (destination_file.exists()):          
            shutil.copy(tif_file, destination_file) 
            df_split_info=analy.generate_dataframe_from_image(image, defect_class, dataframe=df_split_info)         
            print(f"Copied: {image}.tif")
        else:
            print(f"File not found: {image}.tif")    

df_split_info.to_csv(dest_dir.parent/"input_data" /"split_image_info.csv", index=False, encoding='shift_jis', mode='w') 
